In [1]:
!git clone https://github.com/Nik-su/Email_Intent_recognition.git

Cloning into 'Email_Intent_recognition'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 43 (delta 12), reused 35 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (43/43), 1021.06 KiB | 3.19 MiB/s, done.
Resolving deltas: 100% (12/12), done.


In [2]:
%cd Email_Intent_recognition/
!ls

/content/Email_Intent_recognition
 app.py				    model_training.py
'Dataset Preparation Summary.pdf'  'Model Training Summary.pdf'
 dataset.py			    __pycache__
 email_intent_dataset.csv	    Readme.md
 Email_Intent_Run.ipynb		    requirements.txt
 evaluation_model.py		    save_to_colab.py
 Inference_test.py


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 

In [ ]:
!pip install datasets

In [ ]:
!python model_training.py

2025-05-12 15:52:53.689023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747065173.710245    6156 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747065173.716416    6156 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-12 15:52:53.738686: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Running in Google Colab - mounting Google Drive...
Traceback (most recent call last):
  File "/content/Email_Intent_r

In [ ]:
!pip install numpy.rec

ERROR: Could not find a version that satisfies the requirement numpy.rec (from versions: none)
ERROR: No matching distribution found for numpy.rec


In [ ]:
# This script is designed to run ONLY in Google Colab
# Please copy and paste this into a Colab notebook cell

import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from transformers import BertTokenizerFast, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import os
import shutil
from datetime import datetime
import json

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Upload your dataset to Colab if needed
# from google.colab import files
# uploaded = files.upload()  # Uncomment if you need to upload dataset

# Load and prepare data
print("Loading dataset...")
df = pd.read_csv('/content/Email_Intent_recognition/email_intent_dataset.csv')
df = df[['text', 'intent']]
df = df.dropna()

# Encode labels
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['intent'])
label2id = {label: i for i, label in enumerate(label_encoder.classes_)}
id2label = {v: k for k, v in label2id.items()}

# Initialize tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# Create and tokenize dataset
dataset = Dataset.from_pandas(df[['text', 'label']])
dataset = dataset.train_test_split(test_size=0.2)
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

# Initialize model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    save_steps=500
    # evaluation_strategy="epoch",
    # save_strategy="epoch",
    # load_best_model_at_end=True,
)

# Define compute metrics
def compute_metrics(p):
    from sklearn.metrics import accuracy_score, f1_score
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Train model
print("Training model...")
trainer.train()

# Evaluate and save results
predictions = trainer.predict(tokenized_dataset['test'])
y_true = predictions.label_ids
y_pred = np.argmax(predictions.predictions, axis=1)

print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))

# Save model locally first
model.save_pretrained('./results')
tokenizer.save_pretrained('./results')

# Save label mapping
with open('./results/label_mapping.json', 'w') as f:
    json.dump({
        'label2id': label2id,
        'id2label': id2label,
        'label_classes': list(label_encoder.classes_)
    }, f, indent=2)

# Create timestamp for unique folder naming
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"email_intent_model_{timestamp}"

# Save training metadata
metadata = {
    'model_name': model_name,
    'training_timestamp': timestamp,
    'dataset_size': len(df),
    'num_labels': len(label2id),
    'train_size': len(tokenized_dataset['train']),
    'test_size': len(tokenized_dataset['test']),
    'label_classes': list(label_encoder.classes_)
}

with open('./results/training_metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

# Save classification report
classification_report_dict = classification_report(
    y_true, y_pred,
    target_names=label_encoder.classes_,
    output_dict=True
)

with open('./results/classification_report.json', 'w') as f:
    json.dump(classification_report_dict, f, indent=2)

# Save to Google Drive
print("\nSaving to Google Drive...")
gdrive_path = f'/content/drive/MyDrive/email_intent_models/{model_name}'

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(gdrive_path), exist_ok=True)

# Copy the entire results folder to Google Drive
if os.path.exists(gdrive_path):
    shutil.rmtree(gdrive_path)
shutil.copytree('./results', gdrive_path)

print(f"✅ Model saved to Google Drive at: {gdrive_path}")

# Create a quick reference file with latest model info
with open(f'/content/drive/MyDrive/email_intent_models/latest_model.txt', 'w') as f:
    f.write(f"Latest trained model: {model_name}\n")
    f.write(f"Path: {gdrive_path}\n")
    f.write(f"Training completed: {timestamp}\n")
    f.write(f"Accuracy: {classification_report_dict['accuracy']:.4f}\n")
    f.write(f"Number of classes: {len(label_encoder.classes_)}\n")

print(f"\n🎉 Training completed successfully!")
print(f"Model: {model_name}")
print(f"Location: {gdrive_path}")
print(f"Accuracy: {classification_report_dict['accuracy']:.4f}")

Mounted at /content/drive
Loading dataset...


Map:   0%|          | 0/8000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-6-217efb10ec6c>:80: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Training model...


...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nikhilforllms (nikhilforllms-minfy) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
500,0.487800
1000,0.002100
1500,0.000900
2000,0.000600
2500,0.000400
3000,0.000300
3500,0.000200
4000,0.000200
4500,0.000200
5000,0.000200



Classification Report:
                                                          precision    recall  f1-score   support

                            Intent_Amendment_Abstraction       1.00      1.00      1.00       143
                                   Intent_Clause_Protect       1.00      1.00      1.00       123
                                 Intent_Company_research       1.00      1.00      1.00       132
           Intent_Company_research;Intent_Clause_Protect       1.00      1.00      1.00       142
                             Intent_Comparison_LOI_Lease       1.00      1.00      1.00       140
Intent_Comparison_LOI_Lease;Intent_Amendment_Abstraction       1.00      1.00      1.00       136
                                Intent_Lease_Abstraction       1.00      1.00      1.00       129
   Intent_Lease_Abstraction;Intent_Amendment_Abstraction       1.00      1.00      1.00        40
          Intent_Lease_Abstraction;Intent_Clause_Protect       1.00      1.00      1.00      

In [ ]:
!python Inference_test.py

2025-05-12 17:15:07.019238: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747070107.053759   27453 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747070107.063140   27453 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Traceback (most recent call last):
  File "/content/Email_Intent_recognition/Inference_test.py", line 191, in <module>
    drive.mount('/content/drive')
  File "/usr/local/lib/python3.11/dist-packages/google/colab/drive.py", line 100, in mount
    return _mount(
           ^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/drive.py", line 137, in _mount
    _message.blocking_request(
  File "/usr/local/lib/python3.1

In [ ]:
# Interactive inference script for Google Colab
# Use this for testing individual emails interactively

import os
import torch
import json
import pickle
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizerFast
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Function to load model from Google Drive
def load_model_from_gdrive():
    """Load the latest model from Google Drive"""

    # Find latest model
    latest_model_file = '/content/drive/MyDrive/email_intent_models/latest_model.txt'
    models_dir = '/content/drive/MyDrive/email_intent_models'

    model_path = None

    # Try to read latest model info
    if os.path.exists(latest_model_file):
        try:
            with open(latest_model_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.startswith('Path: '):
                        model_path = line.split('Path: ')[1].strip()
                        break
        except:
            pass

    # If not found, look for any model directory
    if model_path is None or not os.path.exists(model_path):
        if os.path.exists(models_dir):
            models = [d for d in os.listdir(models_dir)
                     if os.path.isdir(os.path.join(models_dir, d)) and 'email_intent_model' in d]
            if models:
                models.sort(reverse=True)
                model_path = os.path.join(models_dir, models[0])

    if model_path is None or not os.path.exists(model_path):
        raise Exception("No trained model found in Google Drive")

    print(f"Loading model from: {model_path}")

    # Load model and tokenizer
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizerFast.from_pretrained(model_path)
    model.eval()

    # Load metadata
    is_multi_label = False
    intent_to_id = {}
    id_to_intent = {}
    best_threshold = 0.5

    # Try to load metadata.pkl first
    metadata_path = os.path.join(model_path, 'metadata.pkl')
    if os.path.exists(metadata_path):
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)

        is_multi_label = metadata['is_multi_label']
        intent_to_id = metadata['intent_to_id']
        id_to_intent = metadata['id_to_intent']

        if is_multi_label:
            best_threshold = metadata.get('best_threshold', 0.5)
    else:
        # Try to load label_mapping.json
        label_mapping_path = os.path.join(model_path, 'label_mapping.json')
        if os.path.exists(label_mapping_path):
            with open(label_mapping_path, 'r') as f:
                label_mapping = json.load(f)

            intent_to_id = label_mapping['label2id']
            id_to_intent = label_mapping['id2label']
            # Convert string keys to integers for id_to_intent
            id_to_intent = {int(k): v for k, v in id_to_intent.items()}

    return model, tokenizer, is_multi_label, id_to_intent, best_threshold

# Function to predict intent
def predict_intent(email_text, model, tokenizer, is_multi_label, id_to_intent, threshold=0.5):
    """Predict intent for a given email text"""

    # Tokenize input
    inputs = tokenizer(email_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Get model output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        if is_multi_label:
            # Multi-label prediction
            probs = torch.sigmoid(logits).squeeze()

            predicted_indices = (probs > threshold).nonzero(as_tuple=True)[0]

            # If no intents exceed threshold, take the top one
            if len(predicted_indices) == 0:
                predicted_indices = [torch.argmax(probs).item()]

            # Get predicted intents and their probabilities
            results = []
            for idx in predicted_indices:
                intent_name = id_to_intent[idx.item()]
                probability = probs[idx].item()
                results.append((intent_name, probability))

            # Sort by probability (highest first)
            results.sort(key=lambda x: x[1], reverse=True)

            return results
        else:
            # Single-label prediction
            probs = torch.nn.functional.softmax(logits, dim=-1).squeeze()
            pred_label_id = torch.argmax(probs, dim=0).item()
            confidence = probs[pred_label_id].item()

            predicted_intent = id_to_intent[pred_label_id]

            return [(predicted_intent, confidence)]

# Load model
print("Loading model from Google Drive...")
model, tokenizer, is_multi_label, id_to_intent, best_threshold = load_model_from_gdrive()
print(f"Model loaded successfully!")
print(f"Model type: {'Multi-intent' if is_multi_label else 'Single-intent'}")
print(f"Number of intents: {len(id_to_intent)}")
if is_multi_label:
    print(f"Best threshold: {best_threshold}")

# Test with example emails
test_emails = [
    "Please abstract the lease for the Johnson project (PDF attached). We need to know the base rent, commencement and expiry dates, renewal options, and escalation schedule.",
    "Compare the signed lease to the LOI we submitted last month. I want to know what terms got changed or added, especially around TI allowances.",
    "Could you do a background check on Wexford Corp before we proceed? I'm particularly interested in any public disputes or bankruptcies in the past 5 years."
]

print("\n=== Testing with sample emails ===")
for i, email in enumerate(test_emails, 1):
    print(f"\nTest {i}:")
    print(f"Email: {email[:60]}...")

    results = predict_intent(email, model, tokenizer, is_multi_label, id_to_intent, best_threshold)

    if is_multi_label:
        print("Predicted Intents:")
        for intent, confidence in results:
            print(f"  - {intent}: {confidence:.4f}")
    else:
        intent, confidence = results[0]
        print(f"Predicted Intent: {intent}")
        print(f"Confidence: {confidence:.4f}")

# Interactive prediction function
def predict_email_intent(email_text):
    """Predict intent for any email text"""
    results = predict_intent(email_text, model, tokenizer, is_multi_label, id_to_intent, best_threshold)

    if is_multi_label:
        print("\nPredicted Intents:")
        for intent, confidence in results:
            print(f"  - {intent}: {confidence:.4f}")

        # Format for API output
        intent_strings = [intent for intent, _ in results]
        avg_confidence = np.mean([conf for _, conf in results])
        return ";".join(intent_strings), avg_confidence
    else:
        intent, confidence = results[0]
        print(f"\nPredicted Intent: {intent}")
        print(f"Confidence: {confidence:.4f}")
        return intent, confidence

# Example usage
print("\n" + "="*50)
print("Ready for inference!")
print("="*50)
print("""
To predict intent for any email, use:
    predict_email_intent("Your email text here...")

Example:
    predict_email_intent("Can you extract the key dates from the lease document?")
""")

RuntimeError: Failed to import transformers.models.bert.modeling_bert because of the following error (look up to see its traceback):
Failed to import transformers.generation.utils because of the following error (look up to see its traceback):
No module named 'numpy.strings'

## Inference From GC

In [ ]:
#!/usr/bin/env python3
# Inference.py - Compatible with app.py and loads from Google Drive

import sys
import os
import torch
import json
import pickle
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizerFast

# Mount Google Drive if in Colab
def mount_gdrive():
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        return True
    except ImportError:
        # Not in Colab, assume Google Drive is accessible
        return False
    except Exception as e:
        # Already mounted
        return True

# Check command line arguments
if len(sys.argv) < 2:
    print("ERROR: No email text provided")
    sys.exit(1)

email_text = sys.argv[1]

# Mount Google Drive
mount_gdrive()

# Find the latest model in Google Drive
def find_latest_model():
    """Find the latest trained model in Google Drive"""
    # Check for latest model reference file
    latest_model_file = '/content/drive/MyDrive/email_intent_models/latest_model.txt'
    models_dir = '/content/drive/MyDrive/email_intent_models'

    model_path = None

    # Try to read latest model info
    if os.path.exists(latest_model_file):
        try:
            with open(latest_model_file, 'r') as f:
                lines = f.readlines()
                for line in lines:
                    if line.startswith('Path: '):
                        model_path = line.split('Path: ')[1].strip()
                        break
        except:
            pass

    # If not found, look for any model directory
    if model_path is None or not os.path.exists(model_path):
        if os.path.exists(models_dir):
            models = [d for d in os.listdir(models_dir)
                     if os.path.isdir(os.path.join(models_dir, d)) and 'email_intent_model' in d]
            if models:
                # Get the most recent model (sorted alphabetically, which includes timestamp)
                models.sort(reverse=True)
                model_path = os.path.join(models_dir, models[0])

    return model_path

# Find and load the model
model_path = find_latest_model()

if model_path is None or not os.path.exists(model_path):
    print("ERROR: No trained model found in Google Drive")
    sys.exit(1)

# Load model and tokenizer
try:
    model = BertForSequenceClassification.from_pretrained(model_path)
    tokenizer = BertTokenizerFast.from_pretrained(model_path)
    model.eval()
except Exception as e:
    print(f"ERROR: Could not load model - {e}")
    sys.exit(1)

# Load metadata for label mapping
try:
    # Try to load metadata.pkl first (for multi-intent models)
    metadata_path = os.path.join(model_path, 'metadata.pkl')
    if os.path.exists(metadata_path):
        with open(metadata_path, 'rb') as f:
            metadata = pickle.load(f)

        is_multi_label = metadata['is_multi_label']
        intent_to_id = metadata['intent_to_id']
        id_to_intent = metadata['id_to_intent']

        if is_multi_label:
            best_threshold = metadata.get('best_threshold', 0.5)

    else:
        # Try to load label_mapping.json (for single-intent models)
        label_mapping_path = os.path.join(model_path, 'label_mapping.json')
        if os.path.exists(label_mapping_path):
            with open(label_mapping_path, 'r') as f:
                label_mapping = json.load(f)

            # Assume single-label if no metadata.pkl
            is_multi_label = False
            intent_to_id = label_mapping['label2id']
            id_to_intent = label_mapping['id2label']
            # Convert string keys to integers for id_to_intent
            id_to_intent = {int(k): v for k, v in id_to_intent.items()}
        else:
            print("ERROR: No metadata or label mapping found")
            sys.exit(1)

except Exception as e:
    print(f"ERROR: Could not load metadata - {e}")
    sys.exit(1)

# Predict intent
try:
    # Tokenize input
    inputs = tokenizer(email_text, return_tensors="pt", truncation=True, padding=True, max_length=512)

    # Get model output
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

        if is_multi_label:
            # Multi-label prediction
            probs = torch.sigmoid(logits).squeeze()

            # Use best threshold or default
            threshold = best_threshold if 'best_threshold' in locals() else 0.5
            predicted_indices = (probs > threshold).nonzero(as_tuple=True)[0]

            # If no intents exceed threshold, take the top one
            if len(predicted_indices) == 0:
                predicted_indices = [torch.argmax(probs).item()]

            # Get predicted intents and their probabilities
            predicted_intents = []
            intent_probabilities = []

            for idx in predicted_indices:
                intent_name = id_to_intent[idx.item()]
                probability = probs[idx].item()
                predicted_intents.append(intent_name)
                intent_probabilities.append(probability)

            # Sort by probability (highest first)
            paired_results = list(zip(predicted_intents, intent_probabilities))
            paired_results.sort(key=lambda x: x[1], reverse=True)

            # Format output - join multiple intents with semicolon
            intent_strings = [intent for intent, _ in paired_results]
            result_intent = ";".join(intent_strings)

            # Calculate average confidence for multi-intent
            avg_confidence = np.mean([prob for _, prob in paired_results])

            # Return result in expected format for app.py
            print(f"{result_intent}|{avg_confidence:.4f}")

        else:
            # Single-label prediction
            probs = torch.nn.functional.softmax(logits, dim=-1).squeeze()
            pred_label_id = torch.argmax(probs, dim=0).item()
            confidence = probs[pred_label_id].item()

            # Get predicted intent
            predicted_intent = id_to_intent[pred_label_id]

            # Return result in expected format for app.py
            print(f"{predicted_intent}|{confidence:.4f}")

except Exception as e:
    print(f"ERROR: Prediction failed - {e}")
    sys.exit(1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Intent_Lease_Abstraction|0.8695


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
import subprocess
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import os

# Apply nest_asyncio to allow uvicorn.run inside notebooks or scripts
nest_asyncio.apply()

# Create the FastAPI app
app = FastAPI(title="Email Intent Classification API", version="1.0.0")

# Define input model
class EmailText(BaseModel):
    email_text: str

# Define response model
class PredictionResponse(BaseModel):
    email_text: str
    predicted_intent: str
    confidence: float
    is_multi_intent: bool = False
    all_intents: list = []

# Initialize check - verify Inference.py exists
def check_inference_script():
    """Check if Inference.py exists and is accessible"""
    if not os.path.exists("Inference.py"):
        print("WARNING: Inference.py not found in current directory")
        return False
    return True

# Define the /predict endpoint
@app.post("/predict", response_model=PredictionResponse)
def predict_intent(data: EmailText):
    try:
        # Call Inference.py with the email text as argument
        result = subprocess.run(
            ["python3", "Inference.py", data.email_text],
            capture_output=True,
            text=True,
            check=True
        )

        output = result.stdout.strip()

        # Handle both single and multi-intent outputs
        if "|" in output:
            predicted_intent, confidence = output.split("|")
            confidence = float(confidence)

            # Check if it's multi-intent (contains semicolons)
            if ";" in predicted_intent:
                # Multi-intent case
                intents = predicted_intent.split(";")
                return PredictionResponse(
                    email_text=data.email_text,
                    predicted_intent=predicted_intent,  # Full string with semicolons
                    confidence=confidence,
                    is_multi_intent=True,
                    all_intents=intents
                )
            else:
                # Single intent case
                return PredictionResponse(
                    email_text=data.email_text,
                    predicted_intent=predicted_intent,
                    confidence=confidence,
                    is_multi_intent=False,
                    all_intents=[predicted_intent]
                )
        else:
            return {"error": "Invalid output format from inference script", "raw_output": output}

    except subprocess.CalledProcessError as e:
        error_msg = e.stderr.strip() if e.stderr else "Unknown error"
        return {"error": "Prediction script failed", "details": error_msg}
    except Exception as e:
        return {"error": str(e)}

# Root endpoint
@app.get("/")
def root():
    return {
        "message": "Email Intent Classification API",
        "status": "running",
        "endpoints": {
            "/predict": "POST - Predict intent for email text",
            "/docs": "GET - API documentation",
            "/health": "GET - Health check"
        }
    }

# Health check endpoint
@app.get("/health")
def health_check():
    """Check if the API and inference script are working"""
    try:
        # Test if Inference.py can be called
        result = subprocess.run(
            ["python3", "-c", "import Inference"],
            capture_output=True,
            text=True
        )

        if result.returncode == 0:
            return {
                "status": "healthy",
                "inference_script": "accessible",
                "message": "API is ready to make predictions"
            }
        else:
            return {
                "status": "warning",
                "inference_script": "error",
                "message": "Inference script has issues",
                "details": result.stderr.strip()
            }
    except Exception as e:
        return {
            "status": "error",
            "message": str(e)
        }

# Model info endpoint
@app.get("/model-info")
def model_info():
    """Get information about the current model"""
    try:
        # Try to get model info from Google Drive
        latest_model_file = '/content/drive/MyDrive/email_intent_models/latest_model.txt'
        if os.path.exists(latest_model_file):
            with open(latest_model_file, 'r') as f:
                content = f.read()
            return {"model_info": content, "source": "Google Drive"}
        else:
            return {"message": "No model info available", "source": "Google Drive not accessible"}
    except Exception as e:
        return {"error": str(e)}

# Startup event
@app.on_event("startup")
async def startup_event():
    """Run checks when the API starts"""
    print("🚀 Starting Email Intent Classification API...")

    # Mount Google Drive
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        print("✅ Google Drive mounted successfully")
    except:
        print("⚠️ Google Drive mounting failed or not in Colab")

    # Check if Inference.py exists
    if check_inference_script():
        print("✅ Inference script found")
    else:
        print("❌ Inference script not found")

# Main execution
if __name__ == "__main__":
    # Set ngrok authtoken
    ngrok.set_auth_token("2ljGDXBe5aMRPGsbmCMZCkBXAoB_43ekyReP1bEYvLmLgfcPE")

    # Start ngrok tunnel on port 8000
    public_url = ngrok.connect(8000)
    print("🚀 Your public URL:", public_url)

    # Print API endpoints
    print("\n" + "="*50)
    print("API ENDPOINTS:")
    print("="*50)
    print(f"GET  {public_url}/ - API status")
    print(f"POST {public_url}/predict - Predict email intent")
    print(f"GET  {public_url}/health - Health check")
    print(f"GET  {public_url}/model-info - Current model information")
    print(f"GET  {public_url}/docs - Interactive API documentation")

    # Run the FastAPI app
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

<ipython-input-10-4fb0df585ebc>:144: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


🚀 Your public URL: NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"

API ENDPOINTS:
GET  NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"/ - API status
POST NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"/predict - Predict email intent
GET  NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"/health - Health check
GET  NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"/model-info - Current model information
GET  NgrokTunnel: "https://9659-34-87-66-138.ngrok-free.app" -> "http://localhost:8000"/docs - Interactive API documentation


INFO:     Started server process [6983]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     223.233.70.49:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     223.233.70.49:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     223.233.70.49:0 - "POST /predict HTTP/1.1" 200 OK


In [ ]:
# startup.py - Run this before starting the API

import os
import sys
import subprocess

# Mount Google Drive first
def mount_google_drive():
    """Mount Google Drive in the current session"""
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        print("✅ Google Drive mounted successfully")
        return True
    except Exception as e:
        print(f"⚠️ Google Drive mounting failed: {e}")
        # Check if it's already mounted
        if os.path.exists('/content/drive/MyDrive'):
            print("✅ Google Drive appears to be already mounted")
            return True
        return False

# Check if model exists
def check_model_exists():
    """Check if model exists in Google Drive"""
    latest_model_file = '/content/drive/MyDrive/email_intent_models/latest_model.txt'
    models_dir = '/content/drive/MyDrive/email_intent_models'

    if os.path.exists(latest_model_file):
        print("✅ Latest model reference found")
        return True
    elif os.path.exists(models_dir):
        models = [d for d in os.listdir(models_dir)
                 if os.path.isdir(os.path.join(models_dir, d)) and 'email_intent_model' in d]
        if models:
            print(f"✅ Found {len(models)} model(s) in Google Drive")
            return True

    print("❌ No models found in Google Drive")
    return False

# Test inference script
def test_inference_script():
    """Test if the inference script works"""
    try:
        result = subprocess.run(
            ["python", "Inference_test.py", "Test email for inference"],
            capture_output=True,
            text=True,
            timeout=30
        )

        if result.returncode == 0:
            print("✅ Inference script test successful")
            print(f"Output: {result.stdout.strip()}")
            return True
        else:
            print("❌ Inference script test failed")
            print(f"Error: {result.stderr}")
            return False
    except subprocess.TimeoutExpired:
        print("⚠️ Inference script test timed out")
        return False
    except Exception as e:
        print(f"❌ Error testing inference script: {e}")
        return False

# Main setup function
def setup_api():
    """Setup everything needed for the API"""
    print("🚀 Setting up Email Intent Classification API...")
    print("="*50)

    # Step 1: Mount Google Drive
    print("1. Mounting Google Drive...")
    if not mount_google_drive():
        print("❌ Cannot proceed without Google Drive access")
        return False

    # Step 2: Check model exists
    print("\n2. Checking for trained models...")
    if not check_model_exists():
        print("❌ No trained models found. Please train a model first.")
        return False

    # Step 3: Test inference script
    print("\n3. Testing inference script...")
    if not test_inference_script():
        print("❌ Inference script is not working properly")
        return False

    print("\n" + "="*50)
    print("✅ Setup completed successfully!")
    print("You can now start the API with: python3 app.py")
    return True

if __name__ == "__main__":
    success = setup_api()
    sys.exit(0 if success else 1)

🚀 Setting up Email Intent Classification API...
1. Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully

2. Checking for trained models...
✅ Latest model reference found

3. Testing inference script...
✅ Inference script test successful
Output: Intent_Lease_Listings_Comparison|0.4573

✅ Setup completed successfully!
You can now start the API with: python3 app.py


SystemExit: 0

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import subprocess
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import os
import sys

# Apply nest_asyncio to allow uvicorn.run inside notebooks or scripts
nest_asyncio.apply()

# Create the FastAPI app
app = FastAPI(title="Email Intent Classification API", version="1.0.0")

# Define input model
class EmailText(BaseModel):
    email_text: str

# Define response model
class PredictionResponse(BaseModel):
    email_text: str
    predicted_intent: str
    confidence: float
    is_multi_intent: bool = False
    all_intents: list = []

# Startup checks
@app.on_event("startup")
async def startup_event():
    """Run checks when the API starts"""
    print("🚀 Starting Email Intent Classification API...")

    # Check if Google Drive is accessible
    if not os.path.exists('/content/drive/MyDrive'):
        print("⚠️ Google Drive not accessible. Attempting to mount...")
        try:
            from google.colab import drive
            drive.mount('/content/drive', force_remount=False)
            print("✅ Google Drive mounted successfully")
        except Exception as e:
            print(f"❌ Failed to mount Google Drive: {e}")
    else:
        print("✅ Google Drive is accessible")

    # Check if Inference.py exists
    if not os.path.exists("Inference.py"):
        print("❌ Inference.py not found")
    else:
        print("✅ Inference.py found")

# Define the /predict endpoint
@app.post("/predict", response_model=PredictionResponse)
def predict_intent(data: EmailText):
    try:
        # Set environment variables to reduce verbosity
        env = os.environ.copy()
        env['TF_CPP_MIN_LOG_LEVEL'] = '3'
        env['CUDA_VISIBLE_DEVICES'] = ''  # Force CPU usage to avoid CUDA issues

        # Call Inference.py with the email text as argument
        result = subprocess.run(
            [sys.executable, "Inference.py", data.email_text],
            capture_output=True,
            text=True,
            check=True,
            env=env,
            timeout=30  # 30 second timeout
        )

        output = result.stdout.strip()

        # Handle both single and multi-intent outputs
        if "|" in output:
            predicted_intent, confidence = output.split("|")
            confidence = float(confidence)

            # Check if it's multi-intent (contains semicolons)
            if ";" in predicted_intent:
                # Multi-intent case
                intents = predicted_intent.split(";")
                return PredictionResponse(
                    email_text=data.email_text,
                    predicted_intent=predicted_intent,  # Full string with semicolons
                    confidence=confidence,
                    is_multi_intent=True,
                    all_intents=intents
                )
            else:
                # Single intent case
                return PredictionResponse(
                    email_text=data.email_text,
                    predicted_intent=predicted_intent,
                    confidence=confidence,
                    is_multi_intent=False,
                    all_intents=[predicted_intent]
                )
        else:
            raise HTTPException(
                status_code=500,
                detail=f"Invalid output format from inference script: {output}"
            )

    except subprocess.TimeoutExpired:
        raise HTTPException(
            status_code=408,
            detail="Inference script timed out"
        )
    except subprocess.CalledProcessError as e:
        error_msg = e.stderr.strip() if e.stderr else "Unknown error"
        # Filter out CUDA warnings from error message
        error_lines = error_msg.split('\n')
        filtered_errors = [line for line in error_lines
                          if not any(keyword in line for keyword in ['cuda', 'CUDA', 'cuDNN', 'cuBLAS', 'cuFFT'])]
        filtered_error = '\n'.join(filtered_errors).strip()

        raise HTTPException(
            status_code=500,
            detail=f"Prediction script failed: {filtered_error}"
        )
    except Exception as e:
        raise HTTPException(
            status_code=500,
            detail=str(e)
        )

# Root endpoint
@app.get("/")
def root():
    return {
        "message": "Email Intent Classification API",
        "status": "running",
        "endpoints": {
            "/predict": "POST - Predict intent for email text",
            "/docs": "GET - API documentation",
            "/health": "GET - Health check"
        }
    }

# Health check endpoint
@app.get("/health")
def health_check():
    """Check if the API and inference script are working"""
    checks = {
        "api": "healthy",
        "google_drive": "unknown",
        "inference_script": "unknown",
        "model": "unknown"
    }

    # Check Google Drive
    if os.path.exists('/content/drive/MyDrive'):
        checks["google_drive"] = "accessible"
    else:
        checks["google_drive"] = "not accessible"

    # Check Inference.py
    if os.path.exists("Inference.py"):
        checks["inference_script"] = "found"
    else:
        checks["inference_script"] = "not found"

    # Check if model exists
    if os.path.exists('/content/drive/MyDrive/email_intent_models'):
        checks["model"] = "directory found"
    else:
        checks["model"] = "directory not found"

    # Determine overall status
    if checks["google_drive"] == "accessible" and checks["inference_script"] == "found":
        status = "healthy"
    else:
        status = "degraded"

    return {
        "status": status,
        "checks": checks,
        "message": "Health check completed"
    }

# Model info endpoint
@app.get("/model-info")
def model_info():
    """Get information about the current model"""
    try:
        latest_model_file = '/content/drive/MyDrive/email_intent_models/latest_model.txt'
        if os.path.exists(latest_model_file):
            with open(latest_model_file, 'r') as f:
                content = f.read()
            return {"model_info": content, "source": "Google Drive"}
        else:
            return {"message": "No model info available", "source": "Google Drive not accessible"}
    except Exception as e:
        return {"error": str(e)}

# Main execution
if __name__ == "__main__":
    # Set ngrok authtoken
    ngrok.set_auth_token("2ljGDXBe5aMRPGsbmCMZCkBXAoB_43ekyReP1bEYvLmLgfcPE")

    # Start ngrok tunnel on port 8000
    public_url = ngrok.connect(8000)
    print("🚀 Your public URL:", public_url)

    # Print API endpoints
    print("\n" + "="*50)
    print("API ENDPOINTS:")
    print("="*50)
    print(f"GET  {public_url}/ - API status")
    print(f"POST {public_url}/predict - Predict email intent")
    print(f"GET  {public_url}/health - Health check")
    print(f"GET  {public_url}/model-info - Current model information")
    print(f"GET  {public_url}/docs - Interactive API documentation")

    # Run the FastAPI app
    uvicorn.run("app:app", host="0.0.0.0", port=8000)

<ipython-input-8-3083f4ea33af>:29: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event("startup")


🚀 Your public URL: NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"

API ENDPOINTS:
GET  NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"/ - API status
POST NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"/predict - Predict email intent
GET  NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"/health - Health check
GET  NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"/model-info - Current model information
GET  NgrokTunnel: "https://cfa2-34-125-240-16.ngrok-free.app" -> "http://localhost:8000"/docs - Interactive API documentation


INFO:     Started server process [510]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     223.233.70.49:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     223.233.70.49:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     223.233.70.49:0 - "POST /predict HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [510]
